In [1]:
from transformers import *
import os
import numpy as np
import pandas as pd
import tokenizers

I0528 15:46:12.511733 139972968773376 file_utils.py:41] PyTorch version 1.4.0 available.
I0528 15:46:13.656681 139972968773376 file_utils.py:57] TensorFlow version 2.1.0 available.


In [10]:
model_type = "roberta-base"

tokenizer =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )


def get_prediction_v3(tokenizer, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for word in tweet.split():
        
        sub_words = tokenizer.tokenize(" " + word)
        encoded_word = tokenizer.convert_tokens_to_ids(sub_words)
        orig_word = tokenizer.convert_tokens_to_string(sub_words)
#         print(sub_words, " " + word)
#         print(" " + word, orig_word)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word
        
        start_offsets = cursor
        cursor += len(" " + word)
        end_offsets = cursor
        
        token_level_cursor = start_offsets
        
        for i in range(number_of_tokens):
            
            tweet_offsets.append((start_offsets, end_offsets))
            
            # for bert tokenizer, replace "##" and add " " for first sub_word
            sub_word_len = len(sub_words[i].replace("##", ""))
            if i == 0 and orig_word != " " + word:
                sub_word_len += 1
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            token_level_cursor += sub_word_len
        
#     print(tweet_offsets)
#     print(tweet_offsets_token_level)
        
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
#         print(tweet[offset1: offset2])
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(targets_start, targets_end)
    print(tweet_offsets[targets_start][0], tweet_offsets_token_level[targets_start][0])
    print(tweet_offsets[targets_end][1], tweet_offsets_token_level[targets_end][1])
    
    
    return tweet[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]], \
            tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]

# tweet = "star wars ............ is **** boo??? i wanna do your job hand it over u can act as me at my high school lol"
# selected_text = "l"

tweet = "Happy Mother`s Day to all the Momma`s to be and Mommas currently.  Have a wonderful day!!!"
selected_text = "wonderful"

get_prediction_v3(tokenizer, tweet, selected_text)

73 73
83 83


(' wonderful', ' wonderful')

In [3]:
model_type = "roberta-base"

tokenizer_0 = RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tokenizer_1 = tokenizers.ByteLevelBPETokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tweet = " miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers."


input_ids_orig_0 = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(tweet))

ids_0 = []
offsets_0 = []
offsets_0_token_level = []
cursor = 0


for i, word in enumerate(tweet.lower().split()):
    
    encoded_word = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(" " + word))
    ids_0 += encoded_word
    
    number_of_tokens = len(encoded_word)

    start_offsets = cursor
    cursor += len(" " + word)
    end_offsets = cursor
    
    token_level_cursor = start_offsets

    for i in range(number_of_tokens):
        offsets_0.append((start_offsets, end_offsets))
        curr_token = tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(encoded_word[i]))
        
        offsets_0_token_level.append((token_level_cursor, token_level_cursor + len(curr_token)))
        token_level_cursor += len(curr_token)
    
# print(input_ids_orig_0)
# print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig_0)))
print(ids_0)
print(offsets_0_token_level)
print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(ids_0)))

ids_1 = tokenizer_1.encode(tweet).ids
print(ids_1)
print(tokenizer_1.encode(tweet).offsets)
print(tokenizer_1.decode(ids_1))

[1788, 31, 47, 939, 12905, 119, 11, 2714, 8821, 98, 492, 162, 2710, 9, 2892, 98, 939, 64, 5240, 11, 86, 7, 120, 23, 513, 65, 9, 167, 481, 16328, 4]
[(0, 6), (6, 11), (11, 15), (15, 17), (17, 18), (18, 19), (19, 22), (22, 25), (25, 28), (28, 31), (31, 36), (36, 39), (39, 46), (46, 49), (49, 57), (57, 60), (60, 62), (62, 66), (66, 73), (73, 76), (76, 81), (81, 84), (84, 88), (88, 91), (91, 97), (97, 101), (101, 104), (104, 110), (110, 115), (115, 121), (121, 122)]
 miles from you i`m in essex so give me plenty of warning so i can arrive in time to get at least one of those free beers.
[1788, 31, 47, 939, 12905, 119, 11, 2714, 8821, 98, 492, 162, 2710, 9, 2892, 98, 939, 64, 5240, 11, 86, 7, 120, 23, 513, 65, 9, 167, 481, 16328, 4]
[(0, 6), (6, 11), (11, 15), (15, 17), (17, 18), (18, 19), (19, 22), (22, 25), (25, 28), (28, 31), (31, 36), (36, 39), (39, 46), (46, 49), (49, 57), (57, 60), (60, 62), (62, 66), (66, 73), (73, 76), (76, 81), (81, 84), (84, 88), (88, 91), (91, 97), (97, 101), (10

In [4]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [6]:
# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )

model_type = "roberta-base"

tokenizer =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

for i in range(len(train)):
    
    prediction, prediction_token = get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i])
    prediction = prediction.strip()
    prediction_token = prediction_token.strip()
    print(prediction, "--------------------", prediction_token)
    
#     label = str(train["selected_text"].iloc[i]).lower().strip()
#     if label != prediction:
#         print(
#               label, \
#               "-------------------", \
#               get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]), \
#               "-------------------", train["text"].iloc[i])

i`d have responded, if i were going -------------------- i`d have responded, if i were going
sooo sad -------------------- sooo sad
bullying me... -------------------- bullying me
leave me alone -------------------- leave me alone
sons of ****, -------------------- sons of ****,
http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth -------------------- http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth
fun -------------------- fun
soooo high -------------------- soooo high
both of you -------------------- both of you
wow... u just became cooler. -------------------- wow... u just became cooler.
as much as i love to be hopeful, i reckon the chances are minimal =p i`m never gonna get my cake and stuff -------------------- as much as i love to be hopeful, i reckon the chances are minimal =p i`m never gonna get my cake and stuff
like -------------------- like
dangerously -------------------- dangerously
lost -

i live for pain, bring it on -------------------- i live for pain, bring it on
okay, i`m out for a while back later! -------------------- okay, i`m out for a while back later!
#powerblog what is this powerblog challenge you keep talking about? i`m a newbie follower -------------------- blog what is this powerblog challenge you keep talking about? i`m a newbie follower
died. -------------------- died.
waiting for tish to get off. got to drive my moms crv to pick her up all my myself and duckie. first time -------------------- waiting for tish to get off. got to drive my moms crv to pick her up all my myself and duckie. first time
not going to well! -------------------- not going to well!
going to shower because i don`t want to smell at school tomorrow -------------------- going to shower because i don`t want to smell at school tomorrow
sigh... you know i am... -------------------- sigh... you know i am...
hopefully -------------------- hopefully
here are 4 free twitter tools will get yo

wish i could make it too!!! i hate my commute sometimes -------------------- wish i could make it too!!! i hate my commute sometimes
hopefully i can make some good tips -------------------- hopefully i can make some good tips
and there wasn`t a chance -------------------- and there wasn`t a chance
hates -------------------- hates
thanks for the support -------------------- thanks for the support
there`s really no android twitter app of tweetie`s calibre -------------------- there`s really no android twitter app of tweetie`s calibre
lovely -------------------- lovely
nyappy mother`s day to your mom`s. -------------------- nyappy mother`s day to your mom`s.
yeah.... and im gonna take ur picture off my ipod baby.. -------------------- yeah.... and im gonna take ur picture off my ipod baby..
traumatizing -------------------- traumatizing
lol same here....wish there was a way to microsize everything...lol -------------------- lol same here....wish there was a way to microsize everything...l

oh wow...hope he`s ok u take him 2 the vet? -------------------- oh wow...hope he`s ok u take him 2 the vet?
he died. wait, what about magic jack? i just read it. -------------------- he died. wait, what about magic jack? i just read it.
i`ve just woken up -------------------- i`ve just woken up
love -------------------- love
one of the greatest dinners ever: -------------------- one of the greatest dinners ever:
nooooo its raining......had -------------------- nooooo its raining......
shower. class. more class. taking care of my lady-friend. -------------------- shower. class. more class. taking care of my lady-friend.
great -------------------- great
great -------------------- great
goodmorning twitter, oh my gosh, i woke up soooo nice, lol ... oh hai thar twitterverse. happy #mothersday everybody -------------------- goodmorning twitter, oh my gosh, i woke up soooo nice, lol ... oh hai thar twitterverse. happy #mothersday everybody
i loved -------------------- i loved
hubby went to 

(: i admire you! you`re amazing! you inspire me to write <3 -------------------- : i admire you! you`re amazing! you inspire me to write <3
again...i love being off work -------------------- i love being off work
hahaha, i was busy, now i see what i replied to you! -------------------- hahaha, i was busy, now i see what i replied to you!
thank you -------------------- thank you
very bad. -------------------- very bad.
he`s not logging in yet. and i doubt if he will check his his replies here in twitter. but we must keep on trying! -------------------- he`s not logging in yet. and i doubt if he will check his his replies here in twitter. but we must keep on trying!
very content with my life at the moment -------------------- very content with my life at the moment
thanks, i feel a lot better now.. but it hurt like hell!!! good luck with the lousy student writing. -------------------- thanks, i feel a lot better now.. but it hurt like hell!!! good luck with the lousy student writing.
tha

...

i feel sick after that oreo and cheesecake milkskake lol -------------------- i feel sick after that oreo and cheesecake milkskake lol
(yeah shocked the hell outta me -------------------- yeah shocked the hell outta me
lol. she`s offering me a case of guiness! can you one up her? if you buy me two i`ll stay. let`s start the bidding wars -------------------- lol. she`s offering me a case of guiness! can you one up her? if you buy me two i`ll stay. let`s start the bidding wars
sitting here twittering~~up at 3 am yuk! i want some messages! blue monday!! -------------------- sitting here twittering~~up at 3 am yuk! i want some messages! blue monday!!
dizzy -------------------- dizzy
not good. -------------------- not good.
music soothes the soul... -------------------- music soothes the soul...
people...dont like this part of the job -------------------- dont like this part of the job
: you must rest, don`t work hard how abt cr-chan ? -------------------- : you must rest, don`t work hard h

we forgive you and we have missed you! -------------------- we forgive you and we have missed you!
i miss our walks home together -------------------- i miss our walks home together
haha i am aware of how one contracts a uti. and i have training for work at 8:45 tomorrow morning so i have to go to bed early -------------------- haha i am aware of how one contracts a uti. and i have training for work at 8:45 tomorrow morning so i have to go to bed early
so this my last week of school then finals yesss -------------------- so this my last week of school then finals yesss
woo! -------------------- woo!
so hard... -------------------- so hard...
phootoboothingisfunforbunny -------------------- isfunfor
aw .. the grillz in my background look like plaque .. -------------------- aw .. the grillz in my background look like plaque ..
the construction lookd familar -------------------- the construction lookd familar
they are always excited to go anywhere. -------------------- they are always exc

thanks everyone for all the congrats! -------------------- thanks everyone for all the congrats!
trying to upload one thousand photos!! -------------------- trying to upload one thousand photos!!
good -------------------- good
thanks! we`re still all passing around germs here but we`re surviving. have i missed anything? -------------------- thanks! we`re still all passing around germs here but we`re surviving. have i missed anything?
so now i dont get my car back til monday. boooo -------------------- so now i dont get my car back til monday. boooo
can u add ur link. i can`t find u on youtube. -------------------- can u add ur link. i can`t find u on youtube.
is going to a wedding in the afternoon -------------------- is going to a wedding in the afternoon
the best -------------------- the best
took some phogs -------------------- took some phogs
little girls and their shallowness annoy me. on the good side i didnt straighten my hair at all today and it looks some what decent ---------

.. did you have to bust a cap? tell me you had to bust a cap for jesus! -------------------- .. did you have to bust a cap? tell me you had to bust a cap for jesus!
i wish -------------------- i wish
im kinda bored anyone else i think ill listen to some hip hop its pretty good you should check it out www.bseresults.net -------------------- im kinda bored anyone else i think ill listen to some hip hop its pretty good you should check it out www
i can`t do 30 minutes of treadmill but done 30 minutes for today already, gonna do 20 minutes more -------------------- i can`t do 30 minutes of treadmill but done 30 minutes for today already, gonna do 20 minutes more
im still devastated -------------------- im still devastated
not really just have to install os x on a harddrive and get it to boot on schools imacs. -------------------- not really just have to install os x on a harddrive and get it to boot on schools imacs.
welcomed -------------------- welcomed
haha -------------------- haha
kev

KeyboardInterrupt: 

In [ ]:
tokenizer.tokenize(tweet)

In [ ]:
encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
encoded

In [ ]:
decoded = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded))
decoded

In [ ]:
print(len(encoded), len(tokenizer.tokenize(tweet)))

In [ ]:
tweet.split()

In [ ]:
for (i, word) in enumerate(tweet.split()):
        
    sub_tokens = tokenizer.tokenize(word)

    print(sub_tokens)

In [ ]:
logit_offset = 4
word_level = [(0, 8), (8, 14), (14, 21), (21, 24), (24, 31), (31, 44), (31, 44), (44, 48), (48, 52), (52, 60), (60, 64), (64, 70), (64, 70), (70, 74), (74, 78), (78, 88), (78, 88), (88, 92), (92, 98), (98, 103), (98, 103), (103, 109), (103, 109), (103, 109), (109, 114), (114, 117), (117, 122), (122, 124), (124, 131), (131, 135)]
token_level = [(0, 9), (8, 15), (14, 22), (21, 25), (24, 32), (31, 44), (44, 45), (44, 49), (48, 53), (52, 61), (60, 65), (64, 69), (69, 71), (70, 75), (74, 79), (78, 84), (84, 89), (88, 93), (92, 99), (98, 103), (103, 104), (103, 108), (108, 109), (109, 110), (109, 115), (114, 118), (117, 123), (122, 125), (124, 132), (131, 136)]

word_level = [(0, 0)] * logit_offset + word_level
token_level = [(0, 0)] * logit_offset + token_level

padding_length = 192 - len(word_level)
token_level = token_level + ([(0, 30)] * padding_length)
word_level = word_level + ([(0, 30)] * padding_length)

start_logits = [i / 100 for i in range(192 - 4)]
end_logits = [i / 100 for i in range(192 - 4)]

word_level_bbx = []

prev = word_level[logit_offset]
curr = word_level[logit_offset]
curr_bbx = []


for i in range(len(word_level) - logit_offset):
    
    curr = word_level[i + logit_offset]
    
    if curr[0] < prev[0] and curr[1] > prev[1]:
        break
    
    if curr[0] == prev[0] and curr[1] == prev[1]:
        curr_bbx.append(i)
    else:
        word_level_bbx.append(curr_bbx)
        curr_bbx = [i]
        
    prev = curr

for i in range(len(word_level_bbx)):
    word_level_bbx[i].append(word_level_bbx[i][-1] + 1)
    
start_logits_word_level = [np.max(start_logits[bbx[0]: bbx[-1]]) for bbx in word_level_bbx]
end_logits_word_level = [np.max(end_logits[bbx[0]: bbx[-1]]) for bbx in word_level_bbx]

# start_idx_word_level = np.argmax(start_logits_word_level)
# end_idx_word_level = np.argmax(end_logits_word_level)

In [ ]:
start_logits_word_level